<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/24122018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
print( os.listdir('/content') )

Saving imagenet_class_names.txt to imagenet_class_names (2).txt
User uploaded file "imagenet_class_names.txt" with length 32673 bytes
['.config', 'imagenet_class_names (1).txt', 'imagenet_class_names.txt', 'datasets', 'imagenet_class_names (2).txt', 'sample_data']


In [0]:


#Загрузите последнюю предварительную модель Inception v3: контрольная точка доступна по адресу https://github.com/tensorflow/models/tree/master/research/slim . 
#писок имен классов доступен по адресу https://goo.gl/brXRtZ , но вы должны сначала вставить «background» класс.
import os
import re 
import sys
#import shutil
import tarfile
from six.moves import urllib



CLASS_NAME_REGEX = re.compile(r"^n\d+\s+(.*)\s*$", re.M | re.U)
DATASET_URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"
MODEL_URL = "http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz"



def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()
    

def download_tgz(url):
    file_name=os.path.basename(url)
    name_parts = os.path.split(file_name)
    _, name = name_parts
    name_path = name.rsplit( ".", 2 )[ 0 ]
    path = os.path.join("/content", name_path)
    tgz_path = os.path.join(path, name)
    if os.path.exists(path):
      if os.path.isfile(tgz_path):
          os.remove(tgz_path)
          print("path delete", tgz_path, "Path is:", os.listdir(path), "!")
          
        
    os.makedirs(path, exist_ok=True)
    urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
    tmp_tgz = tarfile.open(tgz_path)
    tmp_tgz.extractall(path=path)
    tmp_tgz.close()
    os.remove(tgz_path)
    print("path created:", path)
    return path
    
# Загружаем модель
MODELS_PATH = download_tgz(MODEL_URL)
# Загружаем датасет
FLOWERS_PATH = download_tgz(DATASET_URL)
 

def load_class_names():
    with open(os.path.join("/content", "imagenet_class_names.txt"), "rb") as f:
        content = f.read().decode("utf-8")
        #sys.stdout.flush()
        return CLASS_NAME_REGEX.findall(content)
      
load_class_names()    
      


In [19]:
#MODEL_URL
import os
name_parts = os.path.splitext(MODEL_URL)  
file, name = name_parts
print(file, name)
name_parts = os.path.split(MODEL_URL)  
file, name = name_parts
print(file, name)
print(name.rsplit( ".", 2 )[ 0 ]) 
print(os.path.dirname(MODEL_URL))

http://download.tensorflow.org/models/inception_v3_2016_08_28.tar .gz
http://download.tensorflow.org/models inception_v3_2016_08_28.tar.gz
inception_v3_2016_08_28
http://download.tensorflow.org/models


In [36]:
MODELS_PATH
FLOWERS_PATH

'/content/flower_photos'

In [0]:
class_names = ["background"] + load_class_names() 

In [38]:
import os
print( os.getcwd() )
print( os.listdir('/content/flower_photos/flower_photos') )

/content
['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses', 'LICENSE.txt']


In [2]:
class_names[:5]

NameError: ignored

In [0]:

#готовим данные
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim
import tensorflow as tf
#from scipy.misc import imresize
from skimage.transform import rescale, resize, downscale_local_mean
import numpy as np
from random import sample
import matplotlib.image as mpimg
import os
MODEL_NAME = "inception_v3.ckpt"

#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/")

width = 299
height = 299
channels = 3
tf.reset_default_graph()

    
n_epochs = 1000
batch_size = 50

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 




#Каждый подкаталог flower_photos каталога содержит все изображения данного класса. Давайте получим список классов:
flowers_root_path = os.path.join(FLOWERS_PATH, "flower_photos")
flower_classes = sorted([dirname for dirname in os.listdir(flowers_root_path)
                  if os.path.isdir(os.path.join(flowers_root_path, dirname))])
#flower_classes


#Давайте получим список всех путей файлов изображений для каждого класса:

from collections import defaultdict

image_paths = defaultdict(list)

for flower_class in flower_classes:
    image_dir = os.path.join(flowers_root_path, flower_class)
    for filepath in os.listdir(image_dir):
        if filepath.endswith(".jpg"):
            image_paths[flower_class].append(os.path.join(image_dir, filepath))
            
            
#Давайте сортируем пути изображения, чтобы заставить этот ноутбук вести себя последовательно на нескольких запусках:

for paths in image_paths.values():
    paths.sort() 

    
#Для получения дополнительных функций манипуляции с изображениями, таких как вращения, проверьте документацию SciPy или эту приятную страницу .



def prepare_image(image, target_width = 299, target_height = 299, max_zoom = 0.2):
    """Zooms and crops the image randomly for data augmentation."""

    
    # Во-первых, давайте найдем самый большой ограничивающий прямоугольник с целевым соотношением размеров, которое вписывается в изображение
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = width if crop_vertically else int(height * target_image_ratio)
    crop_height = int(width / target_image_ratio) if crop_vertically else height
        
   #Теперь давайте уменьшим этот ограничивающий прямоугольник случайным фактором (разделив размеры на случайное число
   # между 1.0 и 1.0 + `max_zoom`
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    # Далее мы можем выбрать случайное место на изображении этого прямоугольника.
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    # Давайте обрезать изображение, используя случайную ограничивающую рамку, которую мы построили.
    image = image[y0:y1, x0:x1]

    # Давайте также перевернем изображение по горизонтали с вероятностью 50% :
    if np.random.rand() < 0.5:
        image = np.fliplr(image)

    # Теперь давайте изменим размер изображения до целевых размеров.
    image = resize(image, (target_width, target_height), mode='reflect')
    
    # Наконец, давайте убедимся, что цвета представлены как
    # 32-бит плавает в диапазоне от 0.0 до 1.0 (на данный момент):
    return image.astype(np.float32) / 255
  

def prepare_batch(flower_paths_and_classes, batch_size):
    batch_paths_and_classes = sample(flower_paths_and_classes, batch_size)
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_batch, y_batch 
  
#раннняя остановка
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)  
  
  #снова получить начальный график v3. На этот раз воспользуемся trainingзаполнителем
  
X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()


#изучить входные данные выходного уровня:
# logits.op.inputs[0].op.inputs[0]  или end_points
end_points["PreLogits"]

#отказаться от 2-го и 3-го измерений с помощью tf.squeeze() функции
prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])

#добавить окончательный полностью подключенный слой поверх этого слоя:
  
n_outputs = len(flower_classes)

with tf.name_scope("new_output_layer"):
    flower_logits = tf.layers.dense(prelogits, n_outputs, name="flower_logits")
    Y_proba = tf.nn.softmax(flower_logits, name="Y_proba")
    
#Добавляемстандвартные вещи

y = tf.placeholder(tf.int32, shape=[None])

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flower_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    flower_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="flower_logits")
    training_op = optimizer.minimize(loss, var_list=flower_vars)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(flower_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() 
    
    
#Разделите свой набор данных на тренировочный набор и тестовый набор. Обучите модель тренировочному набору и оцените его на тестовом наборе.

#Во-первых, мы хотим представить классы как int, а не строки:

flower_class_ids = {flower_class: index for index, flower_class in enumerate(flower_classes)}
#flower_class_ids

#Будет проще перетасовать набор данных, если мы представим его как список пар filepath / class:

flower_paths_and_classes = []
for flower_class, paths in image_paths.items():
    for path in paths:
        flower_paths_and_classes.append((path, flower_class_ids[flower_class]))
        
#Затем давайте перетасовать набор данных и разделим его на обучающий набор и тестовый набор:

test_ratio = 0.2
valid_ratio = 0.4
train_size = int(len(flower_paths_and_classes) * (1 - test_ratio - valid_ratio))
test_size = int(len(flower_paths_and_classes) * test_ratio)
valid_size = int(len(flower_paths_and_classes) * valid_ratio)                
np.random.shuffle(flower_paths_and_classes)


flower_paths_and_classes_test = flower_paths_and_classes[:test_size]
flower_paths_and_classes_valid = flower_paths_and_classes[test_size:valid_size]
flower_paths_and_classes_train = flower_paths_and_classes[valid_size:]

n_iterations_per_epoch = len(flower_paths_and_classes_train) // batch_size

        
n_epochs = 1000
batch_size = 50       

best_loss_val = np.infty
check_interval = 100
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    for epoch in range(n_epochs):
        print("Epoch", epoch, end="")
        for iteration in range(n_iterations_per_epoch):
            print(".", end="")
            X_batch, y_batch = prepare_batch(flower_paths_and_classes_train, batch_size)
            X_test, y_test = prepare_batch(flower_paths_and_classes_test, batch_size=len(flower_paths_and_classes_test)//5)
            X_valid, y_valid = prepare_batch(flower_paths_and_classes_valid, batch_size=len(flower_paths_and_classes_test)//5)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                
                loss_val = loss.eval(feed_dict={X: X_valid, y: y_valid})
                print("  Train loss_val:", loss_val)
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "./my_mnist_model")

INFO:tensorflow:Restoring parameters from /content/inception_v3_2016_08_28/inception_v3.ckpt
Epoch 0.  Train loss_val: 1.805358
...........................................Epoch 0, train accuracy: 16.0000%, valid. accuracy: 12.3288%, valid. best loss: 1.805358
Epoch 1.  Train loss_val: 1.6012149
.....